In [1]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
from pathlib import Path
import requests
from pprint import pprint
from typing import NamedTuple,List
import pickle
import time
from dotenv import load_dotenv
import os
import copy
load_dotenv()


class Card(NamedTuple):
    src: List[str]
    title: str

In [2]:
data_path = (Path.cwd()/'data')
with open(data_path / 'cards.pickle', 'rb') as fp:
    cards = pickle.load(fp)
len(cards)

1740

In [6]:
async def dl_wrapper():
    try:
        with open(data_path / 'card_image_save_index.pickle', 'rb') as fp:
            num_completed = pickle.load(fp)
    except EOFError:
            num_completed = 0
    except FileNotFoundError:
            num_completed = 0

    print(num_completed)
    CARDS_TO_GET = 100
    cards_being_fetched = copy.deepcopy(cards[num_completed:num_completed+CARDS_TO_GET])

    async def get_image(card: Card) -> bytes | None:

        src = card.src

        async with aiohttp.ClientSession() as session:
            async with session.get(src) as response:
                if response.status == 200:
                    # save image
                    # print("FOUND")
                    return await response.read()
                else:
                    return None

    tasks = [asyncio.create_task(get_image(card)) for card in cards_being_fetched]

    completed_tasks = await asyncio.gather(*tasks)

    image_path = Path.cwd() / "images"

    for bd, card in zip(completed_tasks, cards_being_fetched):
        card: Card = card
        if bd == None:
            # SKIP
            continue
        else:
            try:
                with open(image_path / (card.title + '.png'), 'wb') as fp:
                    fp.write(bd)
            except OSError:
                # thrown in filename is invalid for some reason
                # skip
                continue

    # update our current downloaded index
    with open(data_path / 'card_image_save_index.pickle', 'wb') as fp:
        pickle.dump(num_completed+CARDS_TO_GET, fp)
    print(F"--> {num_completed+CARDS_TO_GET}")


for i in range(1):
    print(i)
    await dl_wrapper()
    time.sleep(5)

0
200
--> 300
